# Data Wrangling Exercises

## Acquire (acquire.py)

### Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

**acquire & summarize**

Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
Only include properties that include a latitude and longitude value.
Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.
num_rows_missing	pct_rows_missing
parcelid	0	0.000000
airconditioningtypeid	29041	0.535486
architecturalstyletypeid	54232	0.999982
Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.
num_cols_missing	pct_cols_missing	num_rows
23	38.333	108
24	40.000	123
25	41.667	5280
Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
A dataframe
A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
The output:
The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
Look up the dropna documentation.
You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
Make use of inplace, i.e. inplace=True/False.
Decide how to handle the remaining missing values:

Fill with constant value.
Impute with mean, median, mode.
Drop row/column
wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.

In [2]:
import pandas as pd
import acquire

In [3]:
df = acquire.get_zillow_cluster_data()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
df.shape

(77580, 69)

In [5]:
df.head()


,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [19]:
df['tdate'] = df.transactiondate != df.transactiondate.max()
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,tdate
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,True
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,True
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,True
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,True
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN,True


In [21]:
tdatedf = df[['tdate', 'transactiondate', 'parcelid']]


,tdate,transactiondate,parcelid
0,True,2017-01-01,14297519
1,True,2017-01-01,17052889
2,True,2017-01-01,14186244
3,True,2017-01-01,12177905
4,True,2017-01-01,10887214


In [22]:
tdatedf[tdatedf.tdate == False]

,tdate,transactiondate,parcelid
77579,False,2018-05-25,13083743


In [10]:
#student_df.groupby(['Gender', 'Handed']).Ageyears.agg(['mean', 'median'])
minmaxdate = df.groupby(['parcelid', 'transactiondate']).transactiondate.agg(['min', 'max'])
minmaxdate.shape

(77580, 2)

In [11]:
minmaxdate.head()

,,min,max
parcelid,transactiondate,,
10711855,2017-07-07,2017-07-07,2017-07-07
10711877,2017-08-29,2017-08-29,2017-08-29
10711888,2017-04-04,2017-04-04,2017-04-04
10711910,2017-03-17,2017-03-17,2017-03-17
10711923,2017-03-24,2017-03-24,2017-03-24


In [16]:
df.transactiondate == minmaxdate.max

0        False
1        False
2        False
3        False
4        False
         ...  
77575    False
77576    False
77577    False
77578    False
77579    False
Name: transactiondate, Length: 77580, dtype: bool

Mall Customers

notebook

Acquire data from mall_customers.customers in mysql database.
Summarize data (include distributions and descriptive statistics).
Detect outliers using IQR.
Split data (train-test-split).
Encode categorical columns using a one hot encoder.
Handles missing values.
Scaling
wrangle_mall.py

Acquire data from mall_customers.customers in mysql database.
Split the data
One-hot-encoding
Missing values
Scaling